# Entrenamiento de los modelos

Para ello realizamos una validación cruzado de 5 pasos con los cuatro métodos que se van a usar K-NN, SVM, Naive Bayes, Random forests. Previamente, se habían generado los csv con los conjuntos test y validación que ya han sido procesados previamente.

Primero, creamos los diccionarios necesarios para identificar los archivos de entrenamiento y prueba. Posteriormente, creamos una función matriz que se encargará de llamar al resto de métodos usados para entrenar los modelos.

In [ ]:
import pandas as pd
# Creamos unos diccionarios para identificar los distintos datos de la validación cruzada
transformations = {
    "N" : "norm",
    "O" : "original",
    "S" : "std",
}

methods = {
    "K" : KNN,
    "S" : SVM,
    "R" : RF,
    "N" : NB,
}

def entrenar_modelo(method: str, iteration: int, transformation_type: str, PCA: int): 
    # Obtenemos los conjuntos de entrenamiento y testing de la carpeta datos_validacion_cruzada en función de los parámetros
    # Ejemplo: datos_validacion_cruzada/test1_norm_PCA80.csv o datos_validacion_cruzada/training1_std.csv
    transformacion_str = transfomations[transformation_type]

    # Construimos los nombres de los archivos de entrenamiento y prueba
    if PCA > 0:
        try:
            test_file = f"datos_validacion_cruzada/test{iteration}_{transformacion_str}_PCA{PCA}.csv"
            train_file = f"datos_validacion_cruzada/training{iteration}_{transformacion_str}_PCA{PCA}.csv"
        except FileNotFoundError:
            print(f"Archivos con PCA{PCA} no encontrados, intentando sin PCA...")
            return
    else :
        try:
            test_file = f"datos_validacion_cruzada/test{iteration}_{transformacion_str}.csv"
            train_file = f"datos_validacion_cruzada/training{iteration}_{transformacion_str}.csv"
        except FileNotFoundError:
            print(f"Archivos sin PCA no encontrados.")
            return

    # Guardamos los datos de los archivos en dos dataframes de pandas
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    # Llamamos a la función de entrenamiento correspondiente según el método
    methods[method](df_train, df_test) 

Entrenando SVM
Procemos a entrenar el modelo con el dataset datos_validacion_cruzada/training1_norm.csv y evaluarlo con datos_validacion_cruzada/test1_norm.csv


Una vez tenemos nuestra función matriz, creamos las funciones que se encargan de entrenar entrenar los modelos con los distintos métodos especificos.

In [ ]:
import sklearn

def KNN(train_data, test_data):
    print("Entrenando KNN")

In [ ]:

def SVM(train_data, test_data):
    print("Entrenando SVM")

In [ ]:

def RF(train_data, test_data):
    print("Entrenando Random Forest")

In [ ]:

def NB(train_data, test_data):
    print("Entrenando Naive Bayes")